In [13]:
!pip3 install transformers
!pip3 install torch torchvision
!pip3 install datasets
!pip3 install tqdm
!pip3 install ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import transformers
import torch
import datasets
from datasets import concatenate_datasets, load_dataset
from tqdm.auto import tqdm
from transformers import DataCollatorWithPadding, AdamW
import random

from transformers import AutoTokenizer, AutoModel

In [15]:
# raw_datasets = load_dataset("pile-of-law/pile-of-law", 'us_bills')
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-small-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-small-uncased")


law_list = ['acus_reports', 'cc_casebooks', 'cfpb_creditcard_contracts',
            'ed_policy_guidance', 'eoir', 
            'frcp', 'fre', 'ftc_advisory_opinions', 'hhs_alj_opinions', 
            'uscode']
law_list1 = ['acus_reports', 'cc_casebooks', 'cfpb_creditcard_contracts',
            'ed_policy_guidance', 'eoir', 
            'frcp', 'fre', 'ftc_advisory_opinions', 'hhs_alj_opinions', 
            'uscode', "cfr", "scotus_oral_arguments", "state_codes"]

law_list2 = ['acus_reports', 'cc_casebooks', 'cfpb_creditcard_contracts',
            'ed_policy_guidance', 'eoir', 
            'frcp', 'fre', 'ftc_advisory_opinions', 'hhs_alj_opinions', 
            'uscode', "cfr", "scotus_oral_arguments", "state_codes", "congressional_hearings", "courtlistener_opinions"]


dataset = [load_dataset('pile-of-law/pile-of-law', law, split= 'train') for law in law_list2]

n = len(dataset)
def get_mini_dataset(dataset, subset_size=1):
    subset = [None] * n
    # validation_subset = [] 
    t = []
#     start = time.time()
    for i in range(n):        
        # Append a new subset dataset for the `train` set
        subset[i] = dataset[i].select(range(dataset[i].num_rows//subset_size))
#         t.append(time.time()-start)
#         start = time.time()

    # Create a new DatasetDict with the subset datasets
    a = [i.num_rows//subset_size for i in dataset]
    print(a)
#     print(t)
    merged_dataset = concatenate_datasets(subset)
    subset_dataset = datasets.DatasetDict({'train': merged_dataset})#, 'validation': validation_subset})
    return subset_dataset

mini_raw_datasets = get_mini_dataset(dataset)
print(len(mini_raw_datasets['train']))
print(len(mini_raw_datasets['train']))

text = mini_raw_datasets['train'][:]['text']

bag = [item for sentence in text for item in sentence.replace('\n',' ').split('.') if item != '']
bag_size = len(bag)
print(bag_size)


sentence_a = []
sentence_b = []
label = []

for paragraph in text:
    sentences = [
        sentence for sentence in paragraph.replace('\n','').split('.') if sentence != ''
    ]
    num_sentences = len(sentences)
    length = 25
#     print(num_sentences)
    if num_sentences > 1:
        # while 1:
            start = random.randint(0, num_sentences-2)
            # 50/50 whether is IsNextSentence or NotNextSentence
            if random.random() >= 0.5:
                # this is IsNextSentence
                if len(sentences[start])>length or len(sentences[start+1])>length:
                    sentence_a.append(sentences[start])
                    sentence_b.append(sentences[start+1])
                    label.append(0)
                    # break
            else:
                index = random.randint(0, bag_size-1)
                if len(sentences[start])>length or len(bag[index])>length:
                    # this is NotNextSentence
                    sentence_a.append(sentences[start])
                    sentence_b.append(bag[index])
                    label.append(1)
                    # break

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
batch = tokenizer(sentence_a, sentence_b, max_length=256, padding='max_length', truncation=True, return_tensors='pt')
batch['next_sentence_label'] = torch.LongTensor([label]).T
# batch = tokenizer(sentence_a, sentence_b, truncation=False)


Some weights of the model checkpoint at nlpaueb/legal-bert-small-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:datasets.info:Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/pil

[181, 59, 2023, 507, 1671, 69, 51, 112, 2083, 43, 182, 5997, 157]
13135
13135
71841416


In [16]:
# Retrieve the special token information
special_tokens = tokenizer.special_tokens_map

# Print the special token information
print("Special tokens:")
print(f"   - [CLS] token: {special_tokens['cls_token']}")
print(f"   - [SEP] token: {special_tokens['sep_token']}")
print(f"   - [PAD] token: {special_tokens['pad_token']}")
print(f"   - [UNK] token: {special_tokens['unk_token']}")
# ... (other special tokens if available)

# Print the special token IDs
print("\nSpecial token IDs:")
print(f"   - [CLS] token ID: {tokenizer.convert_tokens_to_ids(special_tokens['cls_token'])}")
print(f"   - [SEP] token ID: {tokenizer.convert_tokens_to_ids(special_tokens['sep_token'])}")
print(f"   - [PAD] token ID: {tokenizer.convert_tokens_to_ids(special_tokens['pad_token'])}")
print(f"   - [UNK] token ID: {tokenizer.convert_tokens_to_ids(special_tokens['unk_token'])}")
print(f"   - [MASK] token ID: {tokenizer.mask_token_id}")
# ... (other special token IDs if available)


Special tokens:
   - [CLS] token: [CLS]
   - [SEP] token: [SEP]
   - [PAD] token: [PAD]
   - [UNK] token: [UNK]

Special token IDs:
   - [CLS] token ID: 101
   - [SEP] token ID: 102
   - [PAD] token ID: 0
   - [UNK] token ID: 100
   - [MASK] token ID: 103


In [17]:
import torch

labels = batch['input_ids']
mask =  batch['attention_mask']
token_type_ids = batch['token_type_ids']
next_sentence_labels = batch['next_sentence_label']
# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 101) * (input_ids != 102)
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 103  # our custom [MASK] token == 3
  


In [18]:
encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels, 'token_type_ids':token_type_ids, "next_sentence_labels": next_sentence_labels}
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [19]:
dataset = Dataset(encodings)

train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [20]:
# # Define dataloader
# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(
#     tokenized_datasets["train"], shuffle=True, batch_size=5, collate_fn=data_collator
# )
# eval_dataloader = DataLoader(
#     tokenized_datasets["validation"], batch_size=5, collate_fn=data_collator
# )

In [21]:
from transformers.models.bert.modeling_bert import BertForPreTraining
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

optim = AdamW(model.parameters(), lr=5e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# activate training mode
model.train()
# initialize optimizer

epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(train_dataloader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        next_sentence_labels = batch['next_sentence_labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels, token_type_ids=token_type_ids, next_sentence_label = next_sentence_labels)
        # extract loss
        loss, prediction_scores = outputs[:2]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/669 [00:00<?, ?it/s]

  0%|          | 0/669 [00:00<?, ?it/s]

In [22]:
# Test simple case
test_text = "The [MASK] is an essential step in the legislative process, allowing \
representatives to propose and debate potential laws before they are enacted."
input_ids = tokenizer(test_text, return_tensors="pt")["input_ids"].to(device)

output = model(input_ids)
prediction_scores = output.prediction_logits
mask_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
mask_scores = prediction_scores[0, mask_index]
sorted_scores, sorted_indices = torch.sort(mask_scores, descending=True)
top_predictions = []
for i in range(len(sorted_scores)):
    token_id = sorted_indices[i].item()
    token = tokenizer.convert_ids_to_tokens(token_id)
    top_predictions.append((token, sorted_scores[i].item()))

print(top_predictions)

[('court', 7.824706554412842), ('legislature', 7.761831760406494), ('process', 6.7205634117126465), ('judiciary', 6.714127540588379), ('statute', 6.539819717407227), ('congress', 6.388943195343018), ('law', 6.129153728485107), ('government', 6.103704452514648), ('vote', 6.076457500457764), ('same', 5.985217094421387), ('time', 5.957887172698975), ('matter', 5.877233982086182), ('that', 5.524561405181885), ('legislation', 5.496363162994385), ('constitution', 5.48394775390625), ('question', 5.399204730987549), ('following', 5.297449588775635), ('board', 5.268397331237793), ('problem', 5.237174987792969), ('idea', 5.177856922149658), ('commission', 5.1466064453125), ('committee', 5.112051010131836), ('transaction', 4.981764793395996), ('role', 4.890422344207764), ('system', 4.86745548248291), ('debate', 4.8628458976745605), ('case', 4.816282272338867), ('result', 4.811273097991943), ('transition', 4.736650466918945), ('decision', 4.6814165115356445), ('job', 4.662008285522461), ('what', 4

In [23]:
# bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
# test_text = "The [MASK] is an essential step in the legislative process, allowing \
# representatives to propose and debate potential laws before they are enacted."
# input_ids = tokenizer(test_text, return_tensors="pt")["input_ids"]
# output = bert_model(input_ids)
# prediction_scores = output.logits
# mask_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
# mask_scores = prediction_scores[0, mask_index]
# sorted_scores, sorted_indices = torch.sort(mask_scores, descending=True)
# top_predictions = []
# for i in range(len(sorted_scores)):
#     token_id = sorted_indices[i].item()
#     token = tokenizer.convert_ids_to_tokens(token_id)
#     top_predictions.append((token, sorted_scores[i].item()))

# print(top_predictions)


In [24]:
# Specify the file path
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

drive_path = '/content/gdrive/My Drive/saved_model/'

file_path = 'model_legalbert_full_more_more_data.pth'

# Save the model
torch.save(model.state_dict(), drive_path + file_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
